# ECON 821 - Sorting Models
### Davis Berlind


In [121]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.stats import norm
from statsmodels.regression.quantile_regression import QuantReg
from sklearn.linear_model import LinearRegression
from linearmodels.iv import IVGMM

np.random.seed(42)

In [20]:
X = pd.read_csv("site_data.csv")
I = pd.read_csv("individual_data.csv")
Z = pd.read_csv("travel_costs.csv")
L = pd.read_csv("site_choice.csv")

# Question 2

## 2.i

In [21]:
N = Z.shape[0]
nlocs = X.shape[0]

choicesets = []
for i in range(N):
    choiceset = X.drop(columns="shares")
    choiceset["cost"] = Z.iloc[i,].values
    choicesets.append(choiceset)
choicesets = np.stack(choicesets)

choices = np.stack([choicesets[i,j,:] for i,j in zip(range(N), L.choice-1)])
        
npar = 6

def NLL(par):
    betas = par[0:npar]
    util = choicesets @ betas
    loglikes = choices @ betas - np.log(np.exp(util).sum(axis=1))
    return -loglikes.sum()

beta0 = np.repeat(0,npar)
results = minimize(NLL, beta0, method = "L-BFGS-B", options={"disp": True})

In [22]:
for val, name in zip(results.x, choiceset.columns):
    print("%s: %f" % (name, val))

ramp: 0.043247
restroom: -0.182959
walleye: 1.736463
salmon: 4.601105
panfish: 0.386033
cost: -0.103054


## 2.ii

In [23]:
choicesets = []
for i in range(N):
    choiceset = X.drop(columns="shares")
    choiceset["cost"] = Z.iloc[i,].values
    choiceset["kids*panfish"] = choiceset.panfish * I.loc[i, "kids"]
    choiceset["kids*restroom"] = choiceset.restroom * I.loc[i, "kids"]
    choiceset["boat*ramp"] = choiceset.ramp * I.loc[i, "boat"]
    choiceset["boat*walleye"] = choiceset.walleye * I.loc[i, "boat"]
    choicesets.append(choiceset)
choicesets = np.stack(choicesets)

choices = np.stack([choicesets[i,j,:] for i,j in zip(range(N), L.choice-1)])

npar = 10

beta0 = np.repeat(0,npar)
results = minimize(NLL, beta0, method = "L-BFGS-B", options={"disp": True})

In [24]:
for val, name in zip(results.x, choiceset.columns):
    print("%s: %f" % (name, val))

ramp: -0.412716
restroom: -0.277471
walleye: 1.297074
salmon: 4.560797
panfish: 0.437940
cost: -0.103054
kids*panfish: -0.162631
kids*restroom: 0.343272
boat*ramp: 0.984083
boat*walleye: 0.698864


## 2.iii

In [28]:
class BLP_Estimator:
    def __init__(self, X, I, Z, L, betas0, thetas0, npar,
                 bnds = None, tolerance = 1e-5, method = "L-BFGS-B"):
        
        self.N = Z.shape[0] # storing number of individuals
        self.nlocs = X.shape[0] # storing number of locations
        
        # creating array of all possible choices for all individuals
        choicesets = []
        for i in range(self.N):
            choiceset = pd.DataFrame({
                "cost" : Z.iloc[i,].values,
                "kids*panfish" : X.panfish * I.loc[i, "kids"],
                "kids*restroom" : X.restroom * I.loc[i, "kids"],
                "boat*ramp" : X.ramp * I.loc[i, "boat"],
                "boat*walleye": X.walleye * I.loc[i, "boat"]
            })
            choicesets.append(choiceset)

        self.choicesets = np.stack(choicesets) 
        self.locs = L.choice-1 # store site choices
        
        # store array of site attributes actually chosenb by each individual
        self.choices = np.stack([self.choicesets[i,j,:] for i,j in zip(range(self.N), self.locs)])
        
        self.shares = X.shares
        self.npar = npar
        self.bnds = bnds
        self.betas = betas0
        self.thetas = thetas0
        self.LL = -np.inf
        self.method = method
        self.tol = tolerance

    def pred_p_j(self, betas):
        """
        Returns predicted shares for all sites given a vector of utility parameters.
        """
        util = self.thetas.transpose() + self.choicesets @ betas
        p_ij = np.exp(util).transpose() / np.exp(util).sum(axis=1)
        return p_ij.sum(axis=1) / self.N

    def theta_update(self, betas):
        """ 
        Returns an updated vector of thetas using a BLP contraction mapping step. 
        """
        p_j = self.pred_p_j(betas)
        return self.thetas + (np.log(self.shares.values) - np.log(p_j))   

    def NLL(self, par):
        """ 
        Performs BLP contraction mapping and returns negative log-likelihood.
        """
        
        betas = par[0:self.npar] # store arbitrary vector of utility coefficients
        d = np.repeat(1,self.nlocs) # initializing some distance
        
        # BLP contraction mapping
        while any(d > self.tol):
            thetas_new = self.theta_update(betas)
            d = np.abs(thetas_new - self.thetas)
            print("D: %.8f     " % (max(d)), end = "\r")           
            self.thetas = thetas_new
            
        self.thetas -= self.thetas[0] # level normalization of thetas
        
        # calculating all possible utilities for all individuals
        util = self.thetas.transpose() + self.choicesets @ betas 
        
        # calculate LL based on actual choice
        loglikes = self.thetas[self.locs] + self.choices @ betas - np.log(np.exp(util).sum(axis=1)) 
        self.LL = loglikes.sum() # store LL incase we need to check
        print("LL: %.3f" %(self.LL), end = "\r")
        return -self.LL

    def MLL(self):
        """
        Performs maximization routine given an optimizer (L-BFGS-B is default). 
        Bounds are optional.
        """
        if self.bnds is None:
            soln = minimize(self.NLL, self.betas, method = self.method, options={"disp": True})
        else:
            soln = minimize(self.NLL, self.betas, method = self.method, bounds = self.bnds, options={"disp": True})
        self.betas = soln.x
        
npar = 5 # hardcode number of utility parameters
tol = 1e-6
# bnds = [(-2,2) for i in range(npar)]

# initialize utility parameters to zero
thetas0 = np.zeros(nlocs) 
betas0 = np.zeros(npar)

BLP_Estimator_2_iii = BLP_Estimator(X, I, Z, L, betas0, thetas0, npar, tolerance = tol)
BLP_Estimator_2_iii.MLL() 

In [29]:
print("Naive BLP Estimates:\n")

names = ["cost", "kids*panfish", "kids*restroom", "boat*ramp", "boat*walleye"] 
print("First-Stage Estimates:")
for val, name in zip(BLP_Estimator_2_iii.betas, names):
    print("%s: %f" % (name, val))
    
lm = LinearRegression()
lm.fit(y=BLP_Estimator_2_iii.thetas, X=X.drop(columns="shares"))
print("\nSecond-Stage Estimates:")
for name, val in zip(X.drop(columns="shares").columns, lm.coef_):
    print("%s: %f" % (name, val))

Naive BLP Estimates:

First-Stage Estimates:
cost: -0.123131
kids*panfish: -0.142550
kids*restroom: 0.617638
boat*ramp: 1.453701
boat*walleye: 0.526558

Second-Stage Estimates:
ramp: -0.533256
restroom: -0.749538
walleye: 2.125778
salmon: 2.375341
panfish: 0.524876


## 2.iv

In [80]:
class Rand_BLP_Estimator(BLP_Estimator):
    def __init__(self, X, I, Z, L, betas0, sigma0, thetas0, npar, randpar, sample_size = 1000,
                 bnds = None, tolerance = 1e-5, method = "L-BFGS-B"):
        super().__init__(X, I, Z, L, betas0, thetas0, npar, bnds, tolerance, method)
        self.nsamp = sample_size
        self.sigma = sigma0
        self.randpar = randpar
    
    def pred_p_j(self, betas_array):
        """
        Returns predicted shares for all sites given a vector of utility parameters.
        """
        p_j = np.zeros(self.nlocs)
        util = self.thetas.reshape((self.nlocs,1)) + self.choicesets @ betas_array
        for i in range(self.N):
            prob = np.exp(util[i,:,:]) / np.exp(util[i,:,:]).sum(axis=0)
            p_j += prob.mean(axis=1)
        p_j /= self.N
        return p_j

    def NLL(self, par):
        """ 
        Performs BLP contraction mapping and returns negative log-likelihood.
        """
        
        betas = par[0:self.npar] # store arbitrary vector of utility coefficients
        sigma = par[self.npar:(self.npar + self.randpar)]
        betas_array = np.repeat(np.reshape(betas,(self.npar,1)), self.nsamp, axis = 1)
        
        # sample random params
        v = np.random.normal(0,sigma,self.nsamp)
        betas_array = betas_array + np.vstack([np.zeros((npar - 1, self.nsamp)), v])
        
        d = np.repeat(1,self.nlocs)     # initializing some distance
        
        # BLP contraction mapping
        while any(d > self.tol):
            thetas_new = self.theta_update(betas_array)
            d = np.abs(thetas_new - self.thetas)
            print("D: %.8f     " % (max(d)), end = "\r")           
            self.thetas = thetas_new
            
        self.thetas -= self.thetas[0] # level normalization of thetas
        
        # calculate LL based on actual choice
        num = self.thetas[self.locs].reshape((self.N,1)) + self.choices @ betas_array
        denom = self.thetas.reshape((self.nlocs,1)) + self.choicesets @ betas_array
        like = np.exp(num) / np.exp(denom).sum(axis=1)
        loglikes = np.log(like.mean(axis=1))
        self.LL = loglikes.sum()

        print("LL: %.3f" %(self.LL), end = "\r")
        return -self.LL
    
    def MLL(self):
        """
        Performs maximization routine given an optimizer (L-BFGS-B is default). 
        Bounds are optional.
        """
        init = np.hstack([self.betas,self.sigma])
        if self.bnds is None:
            soln = minimize(self.NLL, init, method = self.method, options={"disp": True})
        else:
            soln = minimize(self.NLL, init, method = self.method, bounds = self.bnds, options={"disp": True})
        self.betas = soln.x[0:self.npar]
        self.sigma = soln.x[-1]

npar = 5 # hardcode number of utility parameters
tol = 1e-6
bnds = [(None,None) for i in range(npar)]
bnds.append((0,None))

# initialize utility parameters to zero
thetas0 = np.zeros(nlocs) 
betas0 = np.zeros(npar)

# initialize random parameters
randpar = 1
sample_size = 500
sigma0 = 0.1

BLP_Estimator_2_iv = Rand_BLP_Estimator(X, I, Z, L, betas0, sigma0, thetas0, npar, randpar, 
                                        sample_size, bnds, tolerance = tol)
BLP_Estimator_2_iv.MLL() 

In [82]:
print("Naive BLP Estimates:\n")

names = ["cost", "kids*panfish", "kids*restroom", "boat*ramp", "boat*walleye"] 
print("First-Stage Estimates:")
for val, name in zip(BLP_Estimator_2_iv.betas, names):
    print("%s: %f" % (name, val))
print("\nsigma_walleye: %f" % (BLP_Estimator_2_iv.sigma))

lm = LinearRegression()
lm.fit(y=BLP_Estimator_2_iv.thetas, X=X.drop(columns="shares"))
print("\nSecond-Stage Estimates:")
for name, val in zip(X.drop(columns="shares").columns, lm.coef_):
    print("%s: %f" % (name, val))

Naive BLP Estimates:

First-Stage Estimates:
cost: 0.000000
kids*panfish: -0.000000
kids*restroom: -0.000000
boat*ramp: -0.000000
boat*walleye: 0.000000

sigma_walleye: 0.100000

Second-Stage Estimates:
ramp: 0.000381
restroom: 0.102926
walleye: 1.326915
salmon: 3.356123
panfish: 0.179537


## Analysis

### a. 

From each of the models, we see a consistent negative estimate for the marginal utility of travel cost parameter in the range of -0.10 to -0.12, which is consistent with our expectation that sites that are more costly to travel to are less attractive. 

Interestingly, from the first model we see that having a bathroom makes a site less attractive on average, with an estimated marginal disutility of -0.18, almost double the estimate of marginal disutility for travel costs.

In line with what we would expect, better catch rates increase utility across the board. Salmon appears to be the most sought after fish, as a unit increase in the salmon catch rate is associated with an average increase in marginal utility of 4.6. This is followed by walleye (1.7) and panfish (0.39).

Lastly, the first model shows little or no gain in marginal utility from the presence of a ramp at the site.

### b. 

Our un-interacted estimates in the second model are mostly consistent with our estimates from our first model, the one big difference being that having a ramp now decreases the average attractiveness of the site for owners without a boat (-0.41). In the first model having a ramp increased average attractiveness for all owners, though negligibly. This difference is explained by the interaction term of ramp and boat, where we see owners with a boat experience an average increase in marginal utility of 0.1 when they go to sites with ramps (naturally, boat owners need ramps to use their boats). Without accounting for the difference in preferences between boat owners, these differences in marginal utility washed out. 

A similar effect occurs with the estimate for marginal utility from having restrooms. The average marginal disutility for restrooms jumps from -0.18 in the first model to -0.28 in the second model. The first estimate was biased towards zero by not accounting for the benefit fishers with children receive from sites with bathrooms. Having a bathroom increases the average utility for fishers with children by 0.34 (which makes sense, sites with bathrooms are probably more kid friendly in general).

Lastly, we see a positive estimate for the interaction between having a boat and the walleye catch rate (0.7), which tells us fishing for walleye is a more enjoyable experience when in a boat, and a negative estimate for the interaction between the panfish cath rate and having children. This maybe tells us that it is hard for children to catch panfish, so if that is what is available at the site, it might be a worse place to bring children.

### c. 

From the previous two questions, we see that it is not only important to account for observed heterogeneity in the site attributes, but it is also important to account for observed heterogeneity in individuals and how they interact with sites. We also see from the differences in the first two models that without including sources of unobserved heterogeneity, the parameters that we do include in the model will sop up any unobserved differences in sites and individuals. In the next question we see how our estimates differ upon including a source of unobserved site heterogeneity $(\xi)$.

### d. 

After accounting for unobserved site heterogeneity, our qualitative interpretations of the site attributes have not changed (no sign flips occur between the second and the third model). However, the magnitudes of certain effects have changed. 

With regards to individual specific estimates, we see that the average marginal disutility of travel cost has gone from -0.10 to -0.12. We also see that the average marginal utilities for "kids x restroom" and "boat x ramp" have jumped up from 0.34 to 0.62 and from 0.98 to 1.45 respectively, telling us that there is something unobserved at sites with ramps and restrooms that was being captured by these estimates before and biasing them downwards (perhaps more developed sites have worse natural amenities or are more congested). 

For the observable site attributes, after accounting for unobserved site heterogeneity we see a big change in the average disutility from a site having restroom (-0.28 vs. -0.75). This likely indicates that there are beneficial site attributes that are correlated with having a bathroom (e.g. other beneficial amenities that come with increased development, like a tackle shop), which were biasing the average disutility from having a bathroom towards zero. Lastly, we see that a higher catch rate for walleye is now worth more (increase to 2.12 from 1.29), while a higher catch rate for salmon is worth less (decrease from 4.56 to 2.38). So a site having more salmon is correlated with an unobserved and beneficial site attribute, while the opposite is likely true for walleye (maybe salmon tend to live in more naturally beautiful areas, while walleye do not).

# Question 3

## 3.i.a

In [83]:
choicesets = []
for i in range(N):
    choiceset = X.copy()
    choiceset["shares x 100"] = choiceset.shares * 100
    choiceset.drop(columns = "shares", inplace = True)
    choiceset["cost"] = Z.iloc[i,].values
    choicesets.append(choiceset)
choicesets = np.stack(choicesets)

choices = np.stack([choicesets[i,j,:] for i,j in zip(range(N), L.choice-1)])
        
npar = 7

def NLL(par):
    betas = par[0:npar]
    util = choicesets @ betas
    loglikes = choices @ betas - np.log(np.exp(util).sum(axis=1))
    return -loglikes.sum()

beta0 = np.repeat(0,npar)
results = minimize(NLL, beta0, method = "L-BFGS-B", options={"disp": True})

In [84]:
for val, name in zip(results.x, choiceset.columns):
    print("%s: %f" % (name, val))

ramp: -0.009151
restroom: -0.224821
walleye: 0.174673
salmon: 0.240314
panfish: 0.230755
shares x 100: 0.693769
cost: -0.104435


## 3.i.b

In [85]:
choicesets = []
for i in range(N):
    choiceset = X.copy()
    choiceset["shares*100"] = choiceset.shares * 100
    choiceset.drop(columns = "shares", inplace = True)
    choiceset["kids*panfish"] = choiceset.panfish * I.loc[i, "kids"]
    choiceset["kids*restroom"] = choiceset.restroom * I.loc[i, "kids"]
    choiceset["boat*ramp"] = choiceset.ramp * I.loc[i, "boat"]
    choiceset["boat*walleye"] = choiceset.walleye * I.loc[i, "boat"]
    choiceset["cost"] = Z.iloc[i,].values
    choicesets.append(choiceset)
choicesets = np.stack(choicesets)

choices = np.stack([choicesets[i,j,:] for i,j in zip(range(N), L.choice-1)])

npar = 11

beta0 = np.repeat(0,npar)
results = minimize(NLL, beta0, method = "L-BFGS-B", options={"disp": True})

In [86]:
for val, name in zip(results.x, choiceset.columns):
    print("%s: %f" % (name, val))

ramp: -0.508705
restroom: -0.319633
walleye: -0.213645
salmon: 0.159423
panfish: 0.266962
shares*100: 0.695230
kids*panfish: -0.127192
kids*restroom: 0.372497
boat*ramp: 1.048762
boat*walleye: 0.600612
cost: -0.104453


## 3.i.c

In [87]:
print("Naive BLP Estimates:\n")

names = ["cost", "kids*panfish", "kids*restroom", "boat*ramp", "boat*walleye"] 
print("First-Stage Estimates:")
for val, name in zip(BLP_Estimator_2_iii.betas, names):
    print("%s: %f" % (name, val))
    
X_shares = X.copy()
X_shares["shares x 100"] = X_shares.shares * 100

lm = LinearRegression()
lm.fit(y=BLP_Estimator_2_iii.thetas, X=X_shares.drop(columns="shares"))
print("\nSecond-Stage Estimates:")
for name, val in zip(X_shares.drop(columns="shares").columns, lm.coef_):
    print("%s: %f" % (name, val))

Naive BLP Estimates:

First-Stage Estimates:
cost: -0.123131
kids*panfish: -0.142550
kids*restroom: 0.617638
boat*ramp: 1.453701
boat*walleye: 0.526558

Second-Stage Estimates:
ramp: -0.559304
restroom: -0.791782
walleye: 0.991091
salmon: -0.983832
panfish: 0.383274
shares x 100: 0.700870


## Analysis

In each of the three models, we get that an increase in the share of anglers choosing site $j$ is associated with an increase to average marginal utility from choosing site $j$ in the range of 0.6 - 0.7. This counterintuitively indicates an agglomeration effect (having not accounted for the endogeneity of the share choosing each site). 

In the first model, we see a dramatic decrease in the average marginal utility associated with each catch rate. Sites with better catch rates probably attract more anglers, thus increasing the share choosing those sites. So the model thinks more anglers choosing a site explains the appeal of the site, when really it is the site attributes that explain why more anglers choose it, and the former relationship is endogenous. In the second and third models, this effect is even more dramatic with the signs of multiple estimates for the average marginal utility of catch rates flipping negative. The fact that we are only seeing this effect with the estimates for catch rates is an indication that catch rates are the attributes that interact with congestion (anglers are worried about fish being depleted by other anglers, not restrooms or ramp space). 

## 3.ii

In [151]:
choicesets = []
for i in range(N):
    choiceset = X.drop(columns="shares")
    choiceset["cost"] = Z.iloc[i,].values
    choiceset["kids*panfish"] = choiceset.panfish * I.loc[i, "kids"]
    choiceset["kids*restroom"] = choiceset.restroom * I.loc[i, "kids"]
    choiceset["boat*ramp"] = choiceset.ramp * I.loc[i, "boat"]
    choiceset["boat*walleye"] = choiceset.walleye * I.loc[i, "boat"]
    choicesets.append(choiceset)
choicesets = np.stack(choicesets)

choices = np.stack([choicesets[i,j,:] for i,j in zip(range(N), L.choice-1)])

### generating instruments ###
exog = X.copy()
exog["shares x 100"] = exog.shares * 100
exog["constant"] = np.ones(exog.shape[0])

# median regression
qlm = QuantReg(BLP_Estimator_2_iii.thetas, exog.drop(columns="shares"))
instrument = qlm.fit(0.5)

# share intruments
const = instrument.params[-1]
betas = np.concatenate([instrument.params[0:-2], BLP_Estimator_2_iii.betas])
util = const + choicesets @ betas
p_ij = np.exp(util).transpose() / np.exp(util).sum(axis=1)
pred_shares = p_ij.sum(axis=1) / N

### GMM ###
exog = X.drop(columns="shares")
exog["const"] = 1
iv = pd.DataFrame({"shares" : pred_shares * 100})
mod_iv = IVGMM(BLP_Estimator_2_iii.thetas,
                exog,
                X.shares * 100,
                iv)
results_iv = mod_iv.fit(cov_type="unadjusted")
results_iv

IV-GMM Estimation Summary                           
==============================================================================
Dep. Variable:              dependent   R-squared:                     -5.3884
Estimator:                     IV-GMM   Adj. R-squared:                -5.8006
No. Observations:                 100   F-statistic:                    5.6866
Date:                Mon, Dec 16 2019   P-value (F-stat)                0.4592
Time:                        22:18:18   Distribution:                  chi2(6)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
ramp          -0.3715     1.2286    -0.3024     0.7623     -2.7795      2.0364
restroom      -0.4873     0.9756    -0.4994     0.6175     -2.3995      1.4250
walleye        9.1704     4.5635     2.0095     0.0445      0.2261      18.115
salmon         23.230     13.209     1.7587     0.0786     -2.6592      49.120
panfish        1.4040     0.6979     2.0119     0.0442      0.0362      2.7718
const         -1.0433     1.3937    -0.7486     0.4541     -3.7749      1.6882
shares        -4.3513     2.5383    -1.7142     0.0865     -9.3263      0.6238
==============================================================================

Endogenous: shares
Instruments: shares
GMM Covariance
Debiased: False
Unadjusted (Homoskedastic)
IVGMMResults, id: 0x1d7687e3d30

### Analysis

We now observe the expected result, i.e. the estimate on shares x 100 is now negative, indicating a congestion effect. Our estimates on the catch rates for walleyy (9.17), salmon (23.23), and panfish (1.40) are all now significantly larger than before. This is due to the fact that catch rates are probably highly correlated with congestion, so when we weren't accounting for the effect of congestion before the model could not disentangle the beneficial effect of an increase in the catch rates from the correlated increase in congestion costs. Thus, estimates for the average marginal utility from increases to catch rates were being biased towards zero.

# Question 4

## 4.i 

In [317]:
CV_PE = {}

# Partial eq. parameters
alpha = np.abs(BLP_Estimator_2_iii.betas[0])
pe_betas = [param 
            for param, name in zip(results_iv.params, results_iv.params.index) 
            if name != "const"]
pe_betas.extend(BLP_Estimator_2_iii.betas)

#### Scenario A

In [318]:
choicesets = []
for i in range(N):
    choiceset = X.copy()
    choiceset.shares = 100 * choiceset.shares
    choiceset["cost"] = Z.iloc[i,].values
    choiceset["kids*panfish"] = choiceset.panfish * I.loc[i, "kids"]
    choiceset["kids*restroom"] = choiceset.restroom * I.loc[i, "kids"]
    choiceset["boat*ramp"] = choiceset.ramp * I.loc[i, "boat"]
    choiceset["boat*walleye"] = choiceset.walleye * I.loc[i, "boat"]
    choicesets.append(choiceset)
choicesets = np.stack(choicesets)

new_choicesets = []
for i in range(N):
    choiceset = X.copy()
    choiceset.shares = 100 * choiceset.shares
    choiceset.walleye = choiceset.walleye * 1.3
    choiceset["cost"] = Z.iloc[i,].values
    choiceset["kids*panfish"] = choiceset.panfish * I.loc[i, "kids"]
    choiceset["kids*restroom"] = choiceset.restroom * I.loc[i, "kids"]
    choiceset["boat*ramp"] = choiceset.ramp * I.loc[i, "boat"]
    choiceset["boat*walleye"] = choiceset.walleye * I.loc[i, "boat"]
    new_choicesets.append(choiceset)
new_choicesets = np.stack(new_choicesets)

old_util = np.exp(choicesets @ pe_betas).mean(axis=1)
new_util = np.exp(new_choicesets @ pe_betas).mean(axis=1)

CV_PE["Scenario A"] = np.mean(1/alpha * (np.log(new_util) - np.log(old_util)))

#### Scenario B

In [319]:
choicesets_change = []
choicesets_nochange = []

for i in range(N):
    choiceset = X.copy()
    choiceset.shares = 100 * choiceset.shares
    choiceset["cost"] = Z.iloc[i,].values
    choiceset["kids*panfish"] = choiceset.panfish * I.loc[i, "kids"]
    choiceset["kids*restroom"] = choiceset.restroom * I.loc[i, "kids"]
    choiceset["boat*ramp"] = choiceset.ramp * I.loc[i, "boat"]
    choiceset["boat*walleye"] = choiceset.walleye * I.loc[i, "boat"]
    if X.shares[L.choice[i] - 1] > 0.015:
        choicesets_change.append(choiceset)
    else:
        choicesets_nochange.append(choiceset)
        
choicesets_change = np.stack(choicesets_change)
choicesets_nochange = np.stack(choicesets_nochange)

new_choicesets_change = []
new_choicesets_nochange = []

for i in range(N):
    choiceset = X.copy()
    choiceset.shares = 100 * choiceset.shares
    choiceset.walleye = choiceset.walleye * np.where(choiceset.shares > 1.5, 1.3, 1)
    choiceset["cost"] = Z.iloc[i,].values
    choiceset["kids*panfish"] = choiceset.panfish * I.loc[i, "kids"]
    choiceset["kids*restroom"] = choiceset.restroom * I.loc[i, "kids"]
    choiceset["boat*ramp"] = choiceset.ramp * I.loc[i, "boat"]
    choiceset["boat*walleye"] = choiceset.walleye * I.loc[i, "boat"]
    
    if X.shares[L.choice[i] - 1] > 0.015:
        new_choicesets_change.append(choiceset)
    else:
        new_choicesets_nochange.append(choiceset)
        
new_choicesets_change = np.stack(new_choicesets_change)
new_choicesets_nochange = np.stack(new_choicesets_nochange)

old_util = np.exp(choicesets_change @ pe_betas).mean(axis=1)
new_util = np.exp(new_choicesets_change @ pe_betas).mean(axis=1)

CV_PE["Scenario B, Affected"] = np.mean(1/alpha * (np.log(new_util) - np.log(old_util)))

old_util = np.exp(choicesets_nochange @ pe_betas).mean(axis=1)
new_util = np.exp(new_choicesets_nochange @ pe_betas).mean(axis=1)

CV_PE["Scenario B, Unaffected"] = np.mean(1/alpha * (np.log(new_util) - np.log(old_util)))

#### Scenario C

In [320]:
choicesets = []
for i in range(N):
    choiceset = X.copy()
    choiceset.shares = 100 * choiceset.shares
    choiceset["cost"] = Z.iloc[i,].values
    choiceset["kids*panfish"] = choiceset.panfish * I.loc[i, "kids"]
    choiceset["kids*restroom"] = choiceset.restroom * I.loc[i, "kids"]
    choiceset["boat*ramp"] = choiceset.ramp * I.loc[i, "boat"]
    choiceset["boat*walleye"] = choiceset.walleye * I.loc[i, "boat"]
    if X.shares[L.choice[i] - 1] <= 0.015:
        choicesets.append(choiceset)

choicesets = np.stack(choicesets)

new_choicesets = []

for i in range(N):
    choiceset = X.copy()
    choiceset.shares = 100 * choiceset.shares
    choiceset.walleye = choiceset.walleye 
    choiceset["cost"] = Z.iloc[i,].values
    choiceset["kids*panfish"] = choiceset.panfish * I.loc[i, "kids"]
    choiceset["kids*restroom"] = choiceset.restroom * I.loc[i, "kids"]
    choiceset["boat*ramp"] = choiceset.ramp * I.loc[i, "boat"]
    choiceset["boat*walleye"] = choiceset.walleye * I.loc[i, "boat"]
    choiceset = choiceset[choiceset.shares <= 1.5]
    
    if X.shares[L.choice[i] - 1] <= 0.015:
        new_choicesets.append(choiceset)

new_choicesets = np.stack(new_choicesets)

old_util = np.exp(choicesets @ pe_betas).mean(axis=1)
new_util = np.exp(new_choicesets @ pe_betas).mean(axis=1)

CV_PE["Scenario C"] = np.mean(1/alpha * (np.log(new_util) - np.log(old_util)))

#### Scenario D

In [321]:
choicesets_change = []
choicesets_nochange = []

for i in range(N):
    choiceset = X.copy()
    choiceset.shares = 100 * choiceset.shares
    choiceset["cost"] = Z.iloc[i,].values
    choiceset["kids*panfish"] = choiceset.panfish * I.loc[i, "kids"]
    choiceset["kids*restroom"] = choiceset.restroom * I.loc[i, "kids"]
    choiceset["boat*ramp"] = choiceset.ramp * I.loc[i, "boat"]
    choiceset["boat*walleye"] = choiceset.walleye * I.loc[i, "boat"]
    if X.shares[L.choice[i] - 1] > 0.015:
        choicesets_change.append(choiceset)
    else:
        choicesets_nochange.append(choiceset)
        
choicesets_change = np.stack(choicesets_change)
choicesets_nochange = np.stack(choicesets_nochange)

new_choicesets_change = []
new_choicesets_nochange = []

for i in range(N):
    choiceset = X.copy()
    choiceset.shares = 100 * choiceset.shares
    choiceset.walleye = choiceset.walleye 
    choiceset["cost"] = Z.iloc[i,].values + np.where(choiceset.shares > 1.5, 10, 0)
    choiceset["kids*panfish"] = choiceset.panfish * I.loc[i, "kids"]
    choiceset["kids*restroom"] = choiceset.restroom * I.loc[i, "kids"]
    choiceset["boat*ramp"] = choiceset.ramp * I.loc[i, "boat"]
    choiceset["boat*walleye"] = choiceset.walleye * I.loc[i, "boat"]
    
    if X.shares[L.choice[i] - 1] > 0.015:
        new_choicesets_change.append(choiceset)
    else:
        new_choicesets_nochange.append(choiceset)
        
new_choicesets_change = np.stack(new_choicesets_change)
new_choicesets_nochange = np.stack(new_choicesets_nochange)

old_util = np.exp(choicesets_change @ pe_betas).mean(axis=1)
new_util = np.exp(new_choicesets_change @ pe_betas).mean(axis=1)

CV_PE["Scenario D, Affected"] = np.mean(1/alpha * (np.log(new_util) - np.log(old_util)))

old_util = np.exp(choicesets_nochange @ pe_betas).mean(axis=1)
new_util = np.exp(new_choicesets_nochange @ pe_betas).mean(axis=1)

CV_PE["Scenario D, Unaffected"] = np.mean(1/alpha * (np.log(new_util) - np.log(old_util)))

Partial Equilibrium Results

In [322]:
for key in CV_PE.keys():
    print("%s: %.3f" % (key, CV_PE.get(key)))

Scenario A: 7.227
Scenario B, Affected: 0.873
Scenario B, Unaffected: 0.657
Scenario C: 1.521
Scenario D, Affected: -0.113
Scenario D, Unaffected: -0.063


### 4.ii

In [323]:
CV_GE = {}

def pred_shares(choicesets, betas, tolerance):
    N = choicesets.shape[0]
    d = np.ones(choicesets.shape[1])
    while any(d > tolerance):
        util = choicesets @ betas
        p_ij = np.exp(util).transpose() / np.exp(util).sum(axis=1)
        p_j = p_ij.sum(axis=1) / N
        d = np.abs(p_j - choicesets[0,:,5])
        choicesets[:,:,5] = p_j
    return choicesets

### Scenario A

In [324]:
choicesets = []
for i in range(N):
    choiceset = X.copy()
    choiceset.shares = 100 * choiceset.shares
    choiceset["cost"] = Z.iloc[i,].values
    choiceset["kids*panfish"] = choiceset.panfish * I.loc[i, "kids"]
    choiceset["kids*restroom"] = choiceset.restroom * I.loc[i, "kids"]
    choiceset["boat*ramp"] = choiceset.ramp * I.loc[i, "boat"]
    choiceset["boat*walleye"] = choiceset.walleye * I.loc[i, "boat"]
    choicesets.append(choiceset)
choicesets = np.stack(choicesets)

new_choicesets = []
for i in range(N):
    choiceset = X.copy()
    choiceset.shares = 100 * choiceset.shares
    choiceset.walleye = choiceset.walleye * 1.3
    choiceset["cost"] = Z.iloc[i,].values
    choiceset["kids*panfish"] = choiceset.panfish * I.loc[i, "kids"]
    choiceset["kids*restroom"] = choiceset.restroom * I.loc[i, "kids"]
    choiceset["boat*ramp"] = choiceset.ramp * I.loc[i, "boat"]
    choiceset["boat*walleye"] = choiceset.walleye * I.loc[i, "boat"]
    new_choicesets.append(choiceset)

new_choicesets = pred_shares(np.stack(new_choicesets), pe_betas, 1e-7)

old_util = np.exp(choicesets @ pe_betas).mean(axis=1)
new_util = np.exp(new_choicesets @ pe_betas).mean(axis=1)

CV_GE["Scenario A"] = np.mean(1/alpha * (np.log(new_util) - np.log(old_util)))

### Scenario B

In [325]:
choicesets_change = []
choicesets_nochange = []

for i in range(N):
    choiceset = X.copy()
    choiceset.shares = 100 * choiceset.shares
    choiceset["cost"] = Z.iloc[i,].values
    choiceset["kids*panfish"] = choiceset.panfish * I.loc[i, "kids"]
    choiceset["kids*restroom"] = choiceset.restroom * I.loc[i, "kids"]
    choiceset["boat*ramp"] = choiceset.ramp * I.loc[i, "boat"]
    choiceset["boat*walleye"] = choiceset.walleye * I.loc[i, "boat"]
    if X.shares[L.choice[i] - 1] > 0.015:
        choicesets_change.append(choiceset)
    else:
        choicesets_nochange.append(choiceset)
        
choicesets_change = np.stack(choicesets_change)
choicesets_nochange = np.stack(choicesets_nochange)

new_choicesets = []

for i in range(N):
    choiceset = X.copy()
    choiceset.shares = 100 * choiceset.shares
    choiceset.walleye = choiceset.walleye * np.where(choiceset.shares > 1.5, 1.3, 1)
    choiceset["cost"] = Z.iloc[i,].values
    choiceset["kids*panfish"] = choiceset.panfish * I.loc[i, "kids"]
    choiceset["kids*restroom"] = choiceset.restroom * I.loc[i, "kids"]
    choiceset["boat*ramp"] = choiceset.ramp * I.loc[i, "boat"]
    choiceset["boat*walleye"] = choiceset.walleye * I.loc[i, "boat"]
    new_choicesets.append(choiceset)
        
new_choicesets = pred_shares(np.stack(new_choicesets), pe_betas, 1e-7)
change = X.shares[L.choice - 1] > 0.015
notchange = X.shares[L.choice - 1] <= 0.015
new_choicesets_change = new_choicesets[change,:,:]
new_choicesets_nochange = new_choicesets[notchange,:,:]

old_util = np.exp(choicesets_change @ pe_betas).mean(axis=1)
new_util = np.exp(new_choicesets_change @ pe_betas).mean(axis=1)

CV_GE["Scenario B, Affected"] = np.mean(1/alpha * (np.log(new_util) - np.log(old_util)))

old_util = np.exp(choicesets_nochange @ pe_betas).mean(axis=1)
new_util = np.exp(new_choicesets_nochange @ pe_betas).mean(axis=1)

CV_GE["Scenario B, Unaffected"] = np.mean(1/alpha * (np.log(new_util) - np.log(old_util)))

### Scenario C

In [326]:
new_choicesets = []

for i in range(N):
    choiceset = X.copy()
    choiceset.shares = 100 * choiceset.shares
    choiceset.walleye = choiceset.walleye 
    choiceset["cost"] = Z.iloc[i,].values
    choiceset["kids*panfish"] = choiceset.panfish * I.loc[i, "kids"]
    choiceset["kids*restroom"] = choiceset.restroom * I.loc[i, "kids"]
    choiceset["boat*ramp"] = choiceset.ramp * I.loc[i, "boat"]
    choiceset["boat*walleye"] = choiceset.walleye * I.loc[i, "boat"]
    choiceset = choiceset[choiceset.shares <= 1.5]
    new_choicesets.append(choiceset)

new_choicesets = pred_shares(np.stack(new_choicesets), pe_betas, 1e-7)
change = X.shares[L.choice - 1] > 0.015
notchange = X.shares[L.choice - 1] <= 0.015
new_choicesets_change = new_choicesets[change,:,:]
new_choicesets_nochange = new_choicesets[notchange,:,:]

old_util = np.exp(choicesets_change @ pe_betas).mean(axis=1)
new_util = np.exp(new_choicesets_change @ pe_betas).mean(axis=1)

CV_GE["Scenario C, Affected"] = np.mean(1/alpha * (np.log(new_util) - np.log(old_util)))

old_util = np.exp(choicesets_nochange @ pe_betas).mean(axis=1)
new_util = np.exp(new_choicesets_nochange @ pe_betas).mean(axis=1)

CV_GE["Scenario C, Unaffected"] = np.mean(1/alpha * (np.log(new_util) - np.log(old_util)))

### Scenario D

In [327]:
new_choicesets = []

for i in range(N):
    choiceset = X.copy()
    choiceset.shares = 100 * choiceset.shares
    choiceset.walleye = choiceset.walleye 
    choiceset["cost"] = Z.iloc[i,].values + np.where(choiceset.shares > 1.5, 10, 0)
    choiceset["kids*panfish"] = choiceset.panfish * I.loc[i, "kids"]
    choiceset["kids*restroom"] = choiceset.restroom * I.loc[i, "kids"]
    choiceset["boat*ramp"] = choiceset.ramp * I.loc[i, "boat"]
    choiceset["boat*walleye"] = choiceset.walleye * I.loc[i, "boat"]
    new_choicesets.append(choiceset)
        
new_choicesets = np.stack(new_choicesets)

new_choicesets = pred_shares(np.stack(new_choicesets), pe_betas, 1e-7)
change = X.shares[L.choice - 1] > 0.015
notchange = X.shares[L.choice - 1] <= 0.015
new_choicesets_change = new_choicesets[change,:,:]
new_choicesets_nochange = new_choicesets[notchange,:,:]

old_util = np.exp(choicesets_change @ pe_betas).mean(axis=1)
new_util = np.exp(new_choicesets_change @ pe_betas).mean(axis=1)

CV_GE["Scenario D, Affected"] = np.mean(1/alpha * (np.log(new_util) - np.log(old_util)))

old_util = np.exp(choicesets_nochange @ pe_betas).mean(axis=1)
new_util = np.exp(new_choicesets_nochange @ pe_betas).mean(axis=1)

CV_GE["Scenario D, Unaffected"] = np.mean(1/alpha * (np.log(new_util) - np.log(old_util)))

General Equilibrium Results

In [328]:
for key in CV_GE.keys():
    print("%s: %.3f" % (key, CV_GE.get(key)))

Scenario A: 41.686
Scenario B, Affected: 41.856
Scenario B, Unaffected: 38.052
Scenario C, Affected: 23.350
Scenario C, Unaffected: 25.106
Scenario D, Affected: 28.519
Scenario D, Unaffected: 27.464
